In [1]:
push!(LOAD_PATH, "../src")
using VariableHeightInvertedPendulum
using PyCall
using PyPlot
using Polynomials
# using ODE

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
# models = [LIPM(9.8, 1.); PolynomialVariableHeightModel(9.8, Poly([1., 0.2]))]
models = [PolynomialVariableHeightModel(9.8, Poly([1., 0.2, 0.5]))];

In [3]:
test = false
orbital_energies = test ? Dict("negative" => 0.) : Dict("negative" => -0.1, "zero" => 0., "positive" => 0.1)

initial_conditions = Dict{Scenario, Vector{Float64}}()
x0 = -0.3
for model in models
    for kv in orbital_energies
        scenario = Scenario(first(kv), model)
        Eo = last(kv)
        xd0 = velocity_given_orbital_energy(model, x0, Eo)
        initial_conditions[scenario] = [x0; xd0]
    end
end

In [4]:
for kv in initial_conditions
    scenario = first(kv)
    model = scenario.model
    state0 = last(kv)
    sim_movie(model, state0, "$(typeof(model))_E_$(scenario.name).mp4"; test = test)
end

In [5]:
# display_mp4("lipm_E_negative.mp4")